In [1]:
import os

In [2]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/satwik/Downloads/MLproj/airlines_sentiment_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL: str
    local_data_file : Path
    unzip_dir : Path

In [6]:
from airlinesSentiment.constants import *
from airlinesSentiment.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config




In [14]:
import os
import pandas as pd
import gdown
import zipfile
from airlinesSentiment import logger
from airlinesSentiment.utils.common import get_size


[2025-06-26 14:33:58,323: INFO: utils: NumExpr defaulting to 8 threads.]


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self) -> str:
        ''' 
        Fetch data from URL
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e:
            raise e
        
    def extract_zip_files(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory 
        Function returns none
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [13]:
try:
    config = ConfigurationManager()
    data_ingest_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingest_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_files()

except Exception as e:
    raise e

[2025-06-17 16:17:45,891: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-17 16:17:45,961: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-17 16:17:45,972: INFO: common: created directory at: artifacts]
[2025-06-17 16:17:45,973: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-17 16:17:45,974: INFO: 3604886704: Downloading data from https://drive.google.com/file/d/1zR4WXP61yBywRjpHvjjjlAZt-65hgQ89/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1zR4WXP61yBywRjpHvjjjlAZt-65hgQ89
To: /Users/satwik/Downloads/MLproj/airlines_sentiment_classification/artifacts/data_ingestion/data.zip
100%|██████████| 1.13M/1.13M [00:00<00:00, 6.46MB/s]

[2025-06-17 16:17:49,089: INFO: 3604886704: Download data from https://drive.google.com/file/d/1zR4WXP61yBywRjpHvjjjlAZt-65hgQ89/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [15]:
data = pd.read_csv('/Users/satwik/Downloads/MLproj/airlines_sentiment_classification/artifacts/data_ingestion/Tweets.csv')

In [17]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [1]:
import spacy

In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: en-core-web-sm
    Found existing installation: en_core_web_sm 3.8.0
    Uninstalling en_core_web_sm-3.8.0:
      Successfully uninstalled en_core_web_sm-3.8.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import spacy
from spacy.lang.en.examples import sentences

nlp = spacy.load("en_core_web_sm")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)
    


Apple is looking at buying U.K. startup for $1 billion
Apple PROPN nsubj
is AUX aux
looking VERB ROOT
at ADP prep
buying VERB pcomp
U.K. PROPN dobj
startup NOUN dep
for ADP prep
$ SYM quantmod
1 NUM compound
billion NUM pobj
